## Deliverable 2. Create a Customer Travel Destinations Map.

In [32]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [33]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Klaksvik,FO,62.2266,-6.5890,49.12,78,94,28.70,overcast clouds
1,1,Shahreza,IR,32.0339,51.8794,55.83,27,0,5.17,clear sky
2,2,Rikitea,PF,-23.1203,-134.9692,79.65,76,7,8.48,light rain
3,3,Carnarvon,AU,-24.8667,113.6333,77.07,88,5,9.22,clear sky
4,4,Kruisfontein,ZA,-34.0033,24.7314,74.73,70,12,13.62,few clouds


In [34]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [35]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Rikitea,PF,-23.1203,-134.9692,79.65,76,7,8.48,light rain
3,3,Carnarvon,AU,-24.8667,113.6333,77.07,88,5,9.22,clear sky
4,4,Kruisfontein,ZA,-34.0033,24.7314,74.73,70,12,13.62,few clouds
6,6,Bengkulu,ID,-3.8004,102.2655,77.85,87,99,4.54,overcast clouds
7,7,Alta Floresta,BR,-9.8756,-56.0861,87.62,66,40,3.44,scattered clouds


In [36]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City ID                262
City                   262
Country                262
Lat                    262
Lng                    262
Max Temp               262
Humidity               262
Cloudiness             262
Wind Speed             262
Current Description    262
dtype: int64

In [37]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

# I dropped empty rows in step 3 with dropna()

In [38]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,79.65,light rain,-23.1203,-134.9692,
3,Carnarvon,AU,77.07,clear sky,-24.8667,113.6333,
4,Kruisfontein,ZA,74.73,few clouds,-34.0033,24.7314,
6,Bengkulu,ID,77.85,overcast clouds,-3.8004,102.2655,
7,Alta Floresta,BR,87.62,scattered clouds,-9.8756,-56.0861,
9,Butaritari,KI,79.88,light rain,3.0707,172.7902,
12,Piney Green,US,79.02,scattered clouds,34.7160,-77.3202,
14,Avarua,CK,75.25,few clouds,-21.2078,-159.7750,
16,Port Elizabeth,ZA,78.31,few clouds,-33.9180,25.5701,
18,Kirakira,SB,81.23,overcast clouds,-10.4544,161.9205,


In [39]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [40]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,79.65,light rain,-23.1203,-134.9692,People ThankYou
3,Carnarvon,AU,77.07,clear sky,-24.8667,113.6333,Hospitality Carnarvon
4,Kruisfontein,ZA,74.73,few clouds,-34.0033,24.7314,Oyster Bay House Rental
6,Bengkulu,ID,77.85,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
7,Alta Floresta,BR,87.62,scattered clouds,-9.8756,-56.0861,Floresta Amazonica Hotel
...,...,...,...,...,...,...,...
665,Itirapina,BR,82.99,overcast clouds,-22.2528,-47.8228,Pousada Paraíso das Aguas
666,Sinnamary,GF,79.88,broken clouds,5.3833,-52.9500,ALETHEIA Appartement Sinnamary
669,San Cristobal,VE,73.58,overcast clouds,7.7669,-72.2250,Pirineos
676,Parrita,CR,85.19,overcast clouds,9.5201,-84.3225,Beso del Viento


In [41]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [42]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [43]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))